In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df_name = 'nov_nine_var.xlsx'

In [4]:
df = pd.read_excel('./documents/'+df_name).iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [5]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(431, 21)

In [6]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [7]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.math.subtract(y_pred, 0.50384917)
    b = dist

    loss = tf.math.add(a, b)
    return loss

In [8]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(16, activation='relu'))
generator.add(tf.keras.layers.Dense(8, activation='relu'))
generator.add(tf.keras.layers.Dense(4, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='sigmoid'))

generator.compile(loss=my_loss, optimizer='adam')

In [9]:
def return_latest():
    df = pd.read_excel('./documents/'+df_name).iloc[:,1::]
    X = df.iloc[:,0:21]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    latest = X[-1].reshape(1, 21)

    return tf.cast(latest, dtype=tf.float32)

In [10]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    real_data = return_latest()
    loss = my_loss(real_data, X_pred)
    generator.train_on_batch(X_pred, real_data)

    pop = dnn_model.predict(X_pred, verbose=0)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[1.6925428]], pop: [[0.4755039]]
epoch: 1, loss: [[1.691026]], pop: [[0.47494486]]
epoch: 2, loss: [[1.6863554]], pop: [[0.47555187]]
epoch: 3, loss: [[1.6753509]], pop: [[0.475735]]
epoch: 4, loss: [[1.677068]], pop: [[0.47594458]]
epoch: 5, loss: [[1.6759295]], pop: [[0.47580174]]
epoch: 6, loss: [[1.6701505]], pop: [[0.4757473]]
epoch: 7, loss: [[1.6706066]], pop: [[0.47606307]]
epoch: 8, loss: [[1.670325]], pop: [[0.47580773]]
epoch: 9, loss: [[1.6710271]], pop: [[0.47586063]]
epoch: 10, loss: [[1.6720481]], pop: [[0.47578725]]
epoch: 11, loss: [[1.6692994]], pop: [[0.4758551]]
epoch: 12, loss: [[1.6480613]], pop: [[0.47664505]]
epoch: 13, loss: [[1.6643993]], pop: [[0.47620833]]
epoch: 14, loss: [[1.6554286]], pop: [[0.4762038]]
epoch: 15, loss: [[1.6487182]], pop: [[0.47621557]]
epoch: 16, loss: [[1.6541778]], pop: [[0.476249]]
epoch: 17, loss: [[1.6452746]], pop: [[0.47683808]]
epoch: 18, loss: [[1.6522388]], pop: [[0.4758163]]
epoch: 19, loss: [[1.6317505]], po

In [11]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [12]:
print(pop_result.max())
print(pop_result.min())
print(pop_result.mean())

0.50547904
0.47494486
0.4928876


In [13]:
df = np.append(X_result, pop_result, axis=1)

In [14]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/basic_formula.xlsx')